In [14]:
from bayes_opt import BayesianOptimization

# Logging
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

# Loading
from bayes_opt.util import load_logs

##### Step 1: 최적화할 임의의 함수 정의

In [2]:
def black_box(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

In [3]:
black_box(10, 5)

-115

##### Step 2: BayesianOptimization 객체 생성
* 최적화할 함수 정의
* 입력값의 최소값과 최대값 지정

여기서는 두 변수의 범위를 받아서, 해당 범위 안에서 가장 큰 값(maximize)을 리턴하는 x, y값을 찾는다.

In [4]:
# 매개변수의 최소값과 최대값 정의
pbounds = {'x': (2, 4), 'y': (-3, 3)}

# BayesianOptimization 객체 생성
optimizer = BayesianOptimization(
    f=black_box,
    pbounds=pbounds,
    verbose=2,      # 2: 항상출력, 1: 최대값이 관찰된 경우만 출력, 0: 아무 출력없음
    random_state=1,
)

##### Step 3: 옵티마이져로 할 역할 정의
* n_iter: 베이지안 최적화 수행 횟 수(클수록 더 좋은 결과값을 찾음)
* init_points: 무작위 탐색 지점 갯 수(클수록 탐색 공간을 다양화해 성능이 좋아짐)

In [5]:
optimizer.maximize(
    n_iter=3,
    init_points=2
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | -7.135    | 2.834     | 1.322     |
| 2         | -7.78     | 2.0       | -1.186    |
| 3         | -7.11     | 2.218     | -0.7867   |
| 4         | -12.4     | 3.66      | 0.9608    |
| 5         | -6.999    | 2.23      | -0.7392   |


* 최적값 확인

In [17]:
optimizer.max

{'target': -6.999472814518675,
 'params': {'x': 2.2303920156083024, 'y': -0.7392021938893159}}

* 처리결과 출력

In [18]:
for i, res in enumerate(optimizer.res):
    print(f'Iteration {i}: {res}')

Iteration 0: {'target': -7.135455292718879, 'params': {'x': 2.8340440094051482, 'y': 1.3219469606529488}}
Iteration 1: {'target': -7.779531005607566, 'params': {'x': 2.0002287496346898, 'y': -1.1860045642089614}}
Iteration 2: {'target': -7.109925819441113, 'params': {'x': 2.2175526295255183, 'y': -0.7867249801593896}}
Iteration 3: {'target': -12.397162416009818, 'params': {'x': 3.660003815774634, 'y': 0.9608275029525108}}
Iteration 4: {'target': -6.999472814518675, 'params': {'x': 2.2303920156083024, 'y': -0.7392021938893159}}


##### Step 4: 파라메타 값 최소/대값 변경

* 특정 변수의 최소/대값 변경

In [6]:
optimizer.set_bounds(new_bounds={'x': (-2, 3)})

In [7]:
optimizer.maximize(
    n_iter=5,
    init_points=0
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 6         | -2.942    | 1.98      | 0.8567    |
| 7         | -0.4597   | 1.096     | 1.508     |
| 8         | 0.5304    | -0.6807   | 1.079     |
| 9         | -5.33     | -1.526    | 3.0       |
| 10        | -5.419    | -2.0      | -0.5552   |


* 변수를 특정 값으로 지정

In [8]:
optimizer.probe(
    params={'x': 0.5, 'y': 0.7},
    # params=[0.5, 0.7],  # 이렇게 할 수도 있음
    lazy=True
)

In [9]:
optimizer.maximize(init_points=0, n_iter=0)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 11        | 0.66      | 0.5       | 0.7       |


##### Step 5: 로그 저장
일반적으로 객체 선언시 `verbose>0`으로 하여 진행사항을 확인할 수 있지만, 로깅 및 알림에 대한 더 많은 제어가 필요한 경우 `옵저버`를 사용할 수 있음(자세한 사항은 고급편에서 다름).<br/>
여기서는, 간단하게 파일에 진행사항을 저장하고 로드하는 `JSONLogger 객체 사용법`만 설명함.

> 로깅을 위한 `옵저버`의 동작원리:
* 옵저버 인스턴스 생성
* 옵티마이저는 특정 상황을 이벤트로 발송하면 옵저버가 수신함

In [12]:
logger = JSONLogger(path='./logs/basic_logs.log')
# 옵티마이저는 새 매개변수-타겟 조합을 얻을 때 마다 'Events.OPTIMIZATION_STEP' 이벤트를 발생
# 일반적인 로그는 이 정도면 됨.
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

* 로그에 저장

In [13]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 12        | -12.48    | -1.266    | -2.446    |
| 13        | -3.854    | -1.069    | -0.9266   |
| 14        | -3.781    | 0.8835    | 3.0       |
| 15        | 0.2623    | -0.3884   | 0.2339    |
| 16        | 0.8149    | 0.0367    | 1.429     |


##### Step 6: 로그 불러오기
로그를 불러올 때는 새로운 옵티마이저로 불러오며, 옵티마이저를 정의할 때 기존 옵티마이저와 동일하게 작성하지 않아도 됨

In [15]:
new_optimizer = BayesianOptimization(
    f=black_box,
    pbounds={'x': (-2, 2), 'y': (-2, 2)},
    verbose=2,
    random_state=7
)

In [17]:
load_logs(new_optimizer, logs=['./logs/basic_logs.log.json'])

In [18]:
new_optimizer.max

{'target': 0.8148936986448472,
 'params': {'x': 0.036701615946454255, 'y': 1.4286715441244868}}

In [19]:
for i, res in enumerate(new_optimizer.res):
    print(f'Iteration {i}: {res}')

Iteration 0: {'target': -12.4780129010131, 'params': {'x': -1.2662205459144347, 'y': -2.445968431387213}}
Iteration 1: {'target': -3.8540423116243687, 'params': {'x': -1.0686989431116456, 'y': -0.9266356377417138}}
Iteration 2: {'target': -3.7806212120405664, 'params': {'x': 0.8835277086999404, 'y': 3.0}}
Iteration 3: {'target': 0.26231220516598663, 'params': {'x': -0.3883834511552078, 'y': 0.23394119696802412}}
Iteration 4: {'target': 0.8148936986448472, 'params': {'x': 0.036701615946454255, 'y': 1.4286715441244868}}
